In [2]:
#%pip install -q tensorflow-recommenders
import tensorflow as tf
import pandas as pd
from tensorflow.keras.layers import Embedding, Dense
from tensorflow.keras import Sequential
import tensorflow_recommenders as tfrs
from tensorflow.keras.metrics import AUC
import math
import numpy as np



In [3]:
if tf.config.list_physical_devices('GPU'):
    print("GPU is available")
else:
    print("GPU is not available")

GPU is available


In [4]:
# Load data
basepath = '../../datasets/preprocessed_datasets/gabor/'
pandas_interaction_data = pd.read_pickle(basepath + 'user_item_interactions_only_main_products_10k.pkl')
#pandas_interaction_data = pandas_interaction_data.sample(1000)
pandas_interaction_data.amount = pandas_interaction_data.amount.apply(lambda x: 1 if x > 0 else 0) 
product_data = pd.read_pickle(basepath + 'no_product_variants_with_duplicate_ids.pkl')
pandas_interaction_data.main_product_id = pandas_interaction_data.main_product_id.astype(int)
product_data.main_product_id = product_data.main_product_id.astype(int)

display(pandas_interaction_data, product_data)

,customer_id,main_product_id,amount
325791,7347782,7023883,1
133655,4395746,4233880,1
128155,4315328,4234094,1
231063,5839435,5015580,1
42949,2070488,3480813,1
...,...,...,...
187060,5196436,4823122,1
471462,9626549,3846282,1
79554,2204900,4773983,1
158028,4807671,3849864,1


,main_product_id,productNumber,productName,productColorName,articleNumber,brand__id,mainCategory__id,gender,originCountry,line,...,heelHeightGroup,sizeEu,sizeUk,sizeIndex,shaftLength__value,sole,isTransferee,isSuccessor,duplicate_product__ids,product_sizes
0,557559,4058394021466,sportliche Ballerinas Glattleder schwarz,schwarz,02.643.57,6590678,315571,w,PT,F-S|H-W,...,bis 3 cm,40.0,6.5,11.0,0.0,Gummi,0.0,0.0,"[557559, 549733, 549734, 557553, 589488, 58948...","[35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39.0, 40...."
1,551622,4054452768212,elegante Pumps Glattleder schwarz,schwarz,05.160.37,6590677,315573,w,PT,F-S|H-W,...,3 cm - 5 cm,38.0,5.0,8.0,0.0,EVA,0.0,0.0,"[551622, 551505, 551510, 551509, 552973, 55162...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39...."
2,547193,4054452768427,Slipper Glattleder schwarz,schwarz,04.443.27,6590677,315576,w,SK,F-S|H-W,...,3 cm - 5 cm,42.0,8.0,14.0,0.0,PU-TPU,0.0,0.0,"[547193, 547191, 547196, 547187, 547189, 54719...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39...."
3,549776,4059701687894,Sneaker low Rauleder blau,blau,06.968.46,4997827,315567,w,VN,F-S|H-W,...,3 cm - 5 cm,39.0,6.0,10.0,0.0,Gummi-EVA,0.0,0.0,"[549776, 550599, 550596, 550593, 550601, 54978...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39...."
4,550726,4054452851594,elegante Ballerinas Materialmix Lederimitat sc...,schwarz,06.102.67,6590678,315574,w,SK,F-S|H-W,...,bis 3 cm,37.5,4.5,7.0,0.0,EVA,0.0,0.0,"[550726, 550736, 550735, 550732, 550728, 55072...","[35.0, 37.0, 37.5, 38.0, 38.5, 39.0, 40.0, 40...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3292,10603553,4066558951733,Sandale met plateauzool Suède blauw,blauw,24.764.36,6590677,315568,w,SK,F-S,...,5 cm - 8 cm,35.0,2.5,3.0,0.0,PU-TPU,0.0,0.0,"[10603553, 10544662]","[35.0, 37.0]"
3293,10442948,4065171827272,Mokassin Materialmix Leder pink,pink,26.090.21,6590678,315590,w,PT,F-S,...,bis 3 cm,40.5,7.0,12.0,0.0,Gummi,0.0,0.0,"[10442948, 10442949]","[40.5, 41.0]"
3294,10782704,4251234499207,Shopper ANDIE blau,blau,921453,363013,363017,w,DE,F-S,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[10782704],[]
3295,10679703,4066558303617,Elegante pumps Glad leer wit,wit,21.450.60,6590677,315573,w,PT,F-S,...,5 cm - 8 cm,35.0,2.5,3.0,0.0,TPU,0.0,0.0,[10679703],[35.0]


In [5]:
# Create a tf.data.Dataset from the interaction data
interaction_dataset = tf.data.Dataset.from_tensor_slices({
    'customer_id': pandas_interaction_data['customer_id'].values,
    'main_product_id': pandas_interaction_data['main_product_id'].values,
    'amount': pandas_interaction_data['amount'].values
})
#variables 
dataset_len = pandas_interaction_data.shape[0]
test_len = math.ceil(dataset_len * 0.2)
train_len = dataset_len - test_len

metrics_batchsize = 16
train_batch_size = 128
test_batch_size = 64
random_seed = 27

In [6]:
#train test split
tf.random.set_seed(random_seed)
# shuffled = interaction_dataset.shuffle(len, seed=random_seed, reshuffle_each_iteration=False)
# train = shuffled.take(train_len)
# test = shuffled.skip(train_len).take(test_len)
shuffled = interaction_dataset.shuffle(dataset_len, seed=random_seed, reshuffle_each_iteration=False)

train = shuffled.take(train_len)
test = shuffled.skip(train_len).take(test_len)
display(train, test)

<_TakeDataset element_spec={'customer_id': TensorSpec(shape=(), dtype=tf.int64, name=None), 'main_product_id': TensorSpec(shape=(), dtype=tf.int64, name=None), 'amount': TensorSpec(shape=(), dtype=tf.int64, name=None)}>

<_TakeDataset element_spec={'customer_id': TensorSpec(shape=(), dtype=tf.int64, name=None), 'main_product_id': TensorSpec(shape=(), dtype=tf.int64, name=None), 'amount': TensorSpec(shape=(), dtype=tf.int64, name=None)}>

In [7]:
# Unique customer and product identifiers
unique_customer_ids = np.array(pandas_interaction_data["customer_id"].unique())
unique_product_ids = np.array(pandas_interaction_data["main_product_id"].unique())

user_ids_vocabulary = tf.keras.layers.IntegerLookup(mask_token=None)
user_ids_vocabulary.adapt(unique_customer_ids)

product_ids_vocabulary = tf.keras.layers.IntegerLookup(mask_token=None)
product_ids_vocabulary.adapt(unique_product_ids)
# print(unique_product_ids.shape[0], unique_customer_ids.shape[0], unique_product_ids, unique_product_ids)

In [8]:
# Define a model using TensorFlow Recommenders
product_ids_dataset = tf.data.Dataset.from_tensor_slices(unique_product_ids)
class RetailModel(tfrs.Model):

    def __init__(self, unique_product_ids_vocab, unique_customer_ids_vocab, embedding_dimension=32):
        super().__init__()
        # Set up user and product representations
        self.user_embedding = tf.keras.Sequential([
            unique_customer_ids_vocab,
            Embedding(unique_customer_ids_vocab.vocabulary_size(), embedding_dimension)
        ])
        self.product_embedding = tf.keras.Sequential([
            unique_product_ids_vocab,
            Embedding(unique_product_ids_vocab.vocabulary_size(), embedding_dimension)
        ])
        # Set up a dense layer for the task.
        self.dense_layer = Dense(128, activation="relu")
        


        # Set up retrieval task and metrics
        self.task = tfrs.tasks.Retrieval(
            metrics=tfrs.metrics.FactorizedTopK(candidates=product_ids_dataset.batch(metrics_batchsize).map(self.product_model))
        )
        self.auc_metric = AUC(name='auc')
        self.rmse_metric = tf.keras.metrics.RootMeanSquaredError(name='rmse')
        self.precision = tf.keras.metrics.Precision(name='precision')
        self.recall = tf.keras.metrics.Recall(name='recall')

    def product_model(self, product_ids):
        return self.product_embedding(product_ids)

    def dot_product_score(self, user, product):
        """
        Computes the dot product between user and product embeddings to get the interaction score.
        """
        return tf.reduce_sum(user * product, axis=1)

    def compute_loss(self, features, training=False):
        # print(features)
        user_embeddings = self.user_embedding(features["customer_id"])
        product_embeddings = self.product_embedding(features["main_product_id"])
        
        user_output = self.dense_layer(user_embeddings)
        product_output = self.dense_layer(product_embeddings)
        # This is a hypothetical function that returns the logits or scores
        # This needs to be adapted based on your actual model
        positive_logits = self.dot_product_score(user_output, product_output)

        # Update RMSE
        self.rmse_metric.update_state(y_true=features["amount"], y_pred=positive_logits)


        return self.task(user_output, product_output)

    def evaluate(self, validation_dataset, *args, **kwargs):
        # Call the base class's evaluate method
        
    
        # Compute additional metrics
        for features in validation_dataset:
            user_embeddings = self.user_embedding(features["customer_id"])
            product_embeddings = self.product_embedding(features["main_product_id"])
            
            user_output = self.dense_layer(user_embeddings)
            product_output = self.dense_layer(product_embeddings)
            
            positive_logits = self.dot_product_score(user_output, product_output)
            # Assuming you have a binary "label" in your dataset indicating 1 for positive interaction and 0 for negative
            self.precision.update_state(y_true=features["amount"], y_pred=positive_logits)
            self.recall.update_state(y_true=features["amount"], y_pred=positive_logits)
    
        precision_result = self.precision.result().numpy()
        recall_result = self.recall.result().numpy()
    
        # Calculate F1 score
        if (precision_result + recall_result) != 0:
            f1_score = 2 * (precision_result * recall_result) / (precision_result + recall_result)
        else:
            f1_score = 0.0
    
        # Reset the metrics for the next evaluation
        # self.precision.reset_states()
        # self.recall.reset_states()
        base_results = super(RetailModel, self).evaluate(validation_dataset, *args, **kwargs)
    
        return base_results, [precision_result, recall_result, f1_score]  # or append additional results as needed



In [9]:
model = RetailModel(product_ids_vocabulary, user_ids_vocabulary, embedding_dimension=128)
model.compile(optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001)) # using legacy instead of tf.keras.optimizers.Adagrad, because newer version is slow on m1/m2 macs

In [10]:
# Shuffle, batch, and cache the data.
cached_train = train.shuffle(dataset_len).batch(train_batch_size).cache()
cached_test = test.batch(test_batch_size).cache()
# Train the model
# model.fit(cached_train, validation_data=cached_test, validation_freq=5, epochs=3)
model.fit(cached_train, validation_freq=5, epochs=3)

Epoch 1/3
63/63 [==============================] - 23s 352ms/step - auc: 0.0000e+00 - rmse: 0.9950 - precision: 0.0000e+00 - recall: 0.0000e+00 - factorized_top_k/top_1_categorical_accuracy: 0.0367 - factorized_top_k/top_5_categorical_accuracy: 0.0746 - factorized_top_k/top_10_categorical_accuracy: 0.1019 - factorized_top_k/top_50_categorical_accuracy: 0.1972 - factorized_top_k/top_100_categorical_accuracy: 0.2620 - loss: 609.9417 - regularization_loss: 0.0000e+00 - total_loss: 609.9417
Epoch 2/3
63/63 [==============================] - 24s 389ms/step - auc: 0.0000e+00 - rmse: 0.7521 - precision: 0.0000e+00 - recall: 0.0000e+00 - factorized_top_k/top_1_categorical_accuracy: 0.9550 - factorized_top_k/top_5_categorical_accuracy: 0.9754 - factorized_top_k/top_10_categorical_accuracy: 0.9800 - factorized_top_k/top_50_categorical_accuracy: 0.9910 - factorized_top_k/top_100_categorical_accuracy: 0.9942 - loss: 601.9556 - regularization_loss: 0.0000e+00 - total_loss: 601.9556
Epoch 3/3
63/63 

In [11]:
# Evaluate the model
k = 5
result_evaluate_train = model.evaluate(cached_train)
result_evaluate_train

63/63 [==============================] - 32s 499ms/step - auc: 0.0000e+00 - rmse: 7.3301 - precision: 0.0000e+00 - recall: 0.0000e+00 - factorized_top_k/top_1_categorical_accuracy: 1.0000 - factorized_top_k/top_5_categorical_accuracy: 1.0000 - factorized_top_k/top_10_categorical_accuracy: 1.0000 - factorized_top_k/top_50_categorical_accuracy: 1.0000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 280.0592 - regularization_loss: 0.0000e+00 - total_loss: 280.0592


([0.0,
  7.330105781555176,
  0.0,
  0.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  131.88229370117188,
  0,
  131.88229370117188],
 [1.0, 1.0, 1.0])

In [12]:
result_evaluat_test = model.evaluate(cached_test)
result_evaluat_test

32/32 [==============================] - 18s 579ms/step - auc: 0.0000e+00 - rmse: 3.2926 - precision: 0.0000e+00 - recall: 0.0000e+00 - factorized_top_k/top_1_categorical_accuracy: 1.0000 - factorized_top_k/top_5_categorical_accuracy: 1.0000 - factorized_top_k/top_10_categorical_accuracy: 1.0000 - factorized_top_k/top_50_categorical_accuracy: 1.0000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 264.7763 - regularization_loss: 0.0000e+00 - total_loss: 264.7763


([0.0,
  3.2926478385925293,
  0.0,
  0.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  45.47650146484375,
  0,
  45.47650146484375],
 [1.0, 1.0, 1.0])

In [13]:
print("Name\t\tValue")
print("-" * 30)
for metric in model.metrics:
    print(f"{metric.name}\t\t{metric.result().numpy()}")
print(f"AUC: {model.auc_metric.result().numpy()}")

Name		Value
------------------------------
auc		0.0
rmse		3.2926478385925293
precision		0.0
recall		0.0
factorized_top_k/top_1_categorical_accuracy		1.0
factorized_top_k/top_5_categorical_accuracy		1.0
factorized_top_k/top_10_categorical_accuracy		1.0
factorized_top_k/top_50_categorical_accuracy		1.0
factorized_top_k/top_100_categorical_accuracy		1.0
AUC: 0.0


In [14]:
sample = next(iter(cached_test.take(1)))

# Extract customer_id from the sample
customer_id = sample['customer_id'].numpy()
customer_id

array([ 4275416,  5191936,  9181415,  2035007,  8891134, 10874927,
        8610367,  3736938,  1957983,  6480012,  1958913,  8908474,
        7285710,  8867556,  6373394,  2196344,  8446599,  1948734,
        6525510,  7553474,  9446582,  9965105,  2142504, 10860996,
        8121725, 10145415,  9108029, 10511849, 10170223, 10270040,
        6456133,  5530970,  3959734,  7019456,  9736062,  4100439,
        5318358,  9834638,  6798151,  2116409,  2032193,  8118166,
        7410488,  8147869,  9441353,  9971211,  8582644,  8657464,
        6799071, 10017324,  8062560,  4460873,  1992810,  7887224,
        9153335,  2006015,  8272020, 10112803,  6568213,  4336120,
        2083624, 10633283,  2201167,  8185718])

In [15]:
def get_random_customer_from_test_data():
    return next(iter(test.take(1)))['customer_id'].numpy()

def display_item_ids(item_ids):
    display(product_data[product_data.main_product_id.isin(item_ids) ])

def display_products_by_customer_id(customer_id):
    display_item_ids(pandas_interaction_data[pandas_interaction_data.customer_id ==  customer_id].main_product_id.tolist())
    

index = tfrs.layers.factorized_top_k.BruteForce(model.user_embedding, k=5)
index.index_from_dataset(
    product_ids_dataset.batch(100).map(lambda id: (id, model.product_model(id))))

def predict_user(user_id):
    print('predicting user: ', user_id)
    print('user already bought following products: ')
    display_products_by_customer_id(user_id)
    
    score, predicted_product_ids = index(np.array([user_id]))
    
    print('predicted products: ')
    display_item_ids(predicted_product_ids[0].numpy())
    print('scores: ', score[0].numpy())


user_id = get_random_customer_from_test_data()

predict_user(user_id)

predicting user:  4275416
user already bought following products: 


,main_product_id,productNumber,productName,productColorName,articleNumber,brand__id,mainCategory__id,gender,originCountry,line,...,heelHeightGroup,sizeEu,sizeUk,sizeIndex,shaftLength__value,sole,isTransferee,isSuccessor,duplicate_product__ids,product_sizes
512,4134223,4062862442301,Elegante Stiefelette Rauleder gelb,gelb,51.800.13,6590677,5014851,w,SK,H-W,...,3 cm - 5 cm,38.0,5.0,8.0,23.0,EVA,0.0,0.0,"[4134223, 4134220, 4222459, 4134229, 4222456, ...","[35.0, 36.0, 37.0, 37.5, 38.0, 38.5, 39.0, 40...."


predicted products: 


,main_product_id,productNumber,productName,productColorName,articleNumber,brand__id,mainCategory__id,gender,originCountry,line,...,heelHeightGroup,sizeEu,sizeUk,sizeIndex,shaftLength__value,sole,isTransferee,isSuccessor,duplicate_product__ids,product_sizes
2,547193,4054452768427,Slipper Glattleder schwarz,schwarz,04.443.27,6590677,315576,w,SK,F-S|H-W,...,3 cm - 5 cm,42.0,8.0,14.0,0.0,PU-TPU,0.0,0.0,"[547193, 547191, 547196, 547187, 547189, 54719...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39...."
20,464697,4054452768465,Zehentrenner Glattleder weiß,weiß,03.700.21,6590677,315594,w,SK,F-S,...,bis 3 cm,36.0,3.5,4.0,0.0,Gummi,0.0,0.0,"[464697, 464702, 464699, 464700, 464703, 46470...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39...."
146,3505425,4062862137856,Sneaker low Materialmix Leder multicolour,multicolour,56.929.20,4997827,315567,w,VN,H-W,...,3 cm - 5 cm,38.5,5.5,9.0,0.0,Gummi-EVA,0.0,0.0,"[3505425, 3505432, 3505427, 4031321, 4031316, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39...."
645,4775981,4062862770985,Spangenpumps Effektleder beige,beige,62.181.14,6590678,315561,w,SK,F-S,...,5 cm - 8 cm,35.0,2.5,3.0,0.0,TPU,0.0,1.0,"[4775981, 4775992, 4775986, 4775985, 4775989, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39...."
1803,7895710,4064032843970,Eleganter Pumps Textil silber,silber,01.400.60,6590677,315573,w,PT,F-S,...,5 cm - 8 cm,38.0,5.0,8.0,0.0,TPU,1.0,1.0,"[7895710, 7895708, 7895715, 7895707, 7895714, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 40.0, 40...."


scores:  [0.0422136  0.04147534 0.04076851 0.04027575 0.03645492]


In [16]:
def calculate_coverage_and_scores(user_ids, total_items_count):
    all_scores = []
    all_predicted_product_ids = []  # Use a set to track unique recommended items

    for user_id in user_ids:
        # Call the prediction function for each user
        score, predicted_product_ids = index(np.array([user_id]))

        # Append the scores to the all_scores list
        all_scores.extend(score[0].numpy())

        # Update the set of all unique predicted product IDs
        all_predicted_product_ids.extend(predicted_product_ids[0].numpy())

    # Calculate min, max, and average scores
    min_score = np.min(all_scores)
    max_score = np.max(all_scores)
    avg_score = np.mean(all_scores)

    # Calculate item coverage and catalog coverage
    num_unique_items = len(np.unique(all_predicted_product_ids))
    catalog_coverage = num_unique_items / total_items_count

    # Print the results
    print(f"Minimum Score: {min_score}")
    print(f"Maximum Score: {max_score}")
    print(f"Average Score: {avg_score}")
    print(f"Catalog Coverage: {catalog_coverage:.2%} (Unique recommended items {num_unique_items} / Total catalog items {total_items_count})")

    return min_score, max_score, avg_score, catalog_coverage, all_predicted_product_ids

# Example usage
total_items_count = len(pandas_interaction_data.main_product_id.unique())

user_ids = pandas_interaction_data.customer_id.unique()
min_score, max_score, avg_score, catalog_coverage, all_predicted_product_ids = calculate_coverage_and_scores(user_ids, total_items_count)


Minimum Score: 0.028165185824036598
Maximum Score: 0.32150477170944214
Average Score: 0.09805728495121002
Catalog Coverage: 90.56% (Unique recommended items 1247 / Total catalog items 1377)
